In [194]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt


# TP 6

In [195]:
def sepLine(line):
    return line.split(";")[1:]
def getArray(file):
    arr = []
    for line in file:
        line = sepLine(line)
        Line = []
        for x in line:

            x = x.replace(",",".")

            try:
                Line.append(float(x))
            except:
                Line.append(x)
        arr.append(Line)
    return arr


f = "Exo1.csv"
file = open(f,"r")

array = getArray(file)
array = array[1:]

In [196]:
class Knn:
    def __init__(self,data):
        self.data = data

    def manhattan(self,index=None,data=None):
        if data:
            element = data
        else:
            element = self.data[index]
        distances = {}
        for i in range(len(self.data)):
            somme=0
            for j in range(len(self.data[i]) - 1):
                somme += abs(element[j]-self.data[i][j])
            distances[i]=somme
        return distances
    
    def euclidienne(self,index=None,data=None):
        if data:
            element = data
        else:
            element = self.data[index]

        distances = {}
        for i in range(len(self.data)):
            somme=0
            for j in range(len(self.data[i]) - 1):
                somme += pow(element[j]-self.data[i][j],2)
            distances[i]=sqrt(somme)
        return distances
    
    def minkowski(self,index=None,p=1,data=None):
        if data:
            element = data
        else:
            element = self.data[index]

        distances = {}
        for i in range(len(self.data)):
            somme=0
            for j in range(len(self.data[i])-1):
                somme += pow(abs(element[j]-self.data[i][j]),p)
            distances[i]=pow(somme,1/p)
        return distances 
    
    def cosine(self,index=None,data=None):
        if data:
            element = data
        else:
            element = self.data[index]

        distances = {}
        for i in range(len(self.data)):
            somme=0
            ab = 0
            a2 = 0
            b2 = 0
            for j in range(len(self.data[i])-1):
                ab += self.data[i][j] * element[j]
                a2 += pow(self.data[i][j],2)
                b2 += pow(element[j],2)

            somme = ab/(sqrt(a2)*sqrt(b2))
            distances[i]= 1 - somme
        return distances
        pass
    
    def hamming(self,index=None,data=None):
        if data:
            element = data
        else:
            element = self.data[index]

        distances = {}
        for i in range(len(self.data)):
            somme=0
            for j in range(len(self.data[i]) - 1):
                if element[j] != self.data[i][j]:
                    somme+=1
            distances[i]=somme
    
        return distances

    def getK(self,distances,k):
        return sorted(distances.items(), key=lambda t: t[1])[:k]
    
    def getClass(self,distances):
        classes = {}
        for (i,dist) in distances:

            if self.data[i][-1] in classes:
                classes[self.data[i][-1]] += 1
            else:
                classes[self.data[i][-1]] = 1

        return classes

In [197]:
import random


class kmeans:
    def __init__(self,pdata,K):
        self.K = K
        self.pdata = pdata
        self.centroids = []
        self.datas = {}
        for i in range(self.K):
            self.datas[i] = []

    def init_centroids(self):
        tier = int(self.pdata.shape[0]/3)
        for i in range(self.K):
            self.centroids.append(list(self.pdata.iloc[i*tier]))

    def init_data(self):
        for i in range(self.pdata.shape[0]):
            row = list(self.pdata.iloc(i))
            self.datas[self.getCluster(row)].append(row)

    def centroid(self,data):
        return data.mean()
    
    def updateCentroid(self):
        self.centroids = []
        for data in self.datas:
            self.centroids.append(self.centroid(pd.DataFrame(data)))

    def addToDatas(self,item):
        self.datas[self.getCluster(item)].append(item)

    def getCluster(self,item):
        self.Knn = Knn(self.centroids)
        return int(list(self.Knn.getClass(self.Knn.getK(self.Knn.manhattan(data=item),1)).keys())[0])
    
    def updateDatas(self):
        for key in self.datas.keys():
            for elem in self.datas[key]:
                newCluster = self.getCluster(elem)
                if newCluster != key:
                    self.datas[key].remove(elem)
                    self.datas[newCluster].append(elem)
    
    def run(self,iter=10):
        self.init_centroids()
        self.init_data()
        for _ in range(iter):
            self.updateCentroid()
            self.updateDatas()


In [198]:
Data = pd.read_excel("Exo1.xlsx")
Data = Data.drop(columns=Data.columns[0])
items = Data["Class"].unique()

for i,txt in enumerate(items):
    Data["Class"] = Data["Class"].replace(txt,i)

In [199]:
kms = kmeans(pdata=Data,K=3)
kms.run()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa\n
1,4.9,3.0,1.4,0.2,Iris-setosa\n
2,4.7,3.2,1.3,0.2,Iris-setosa\n
3,4.6,3.1,1.5,0.2,Iris-setosa\n
4,5.0,3.6,1.4,0.2,Iris-setosa\n
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica\n
146,6.3,2.5,5.0,1.9,Iris-virginica\n
147,6.5,3.0,5.2,2.0,Iris-virginica\n
148,6.2,3.4,5.4,2.3,Iris-virginica\n


In [ ]:
kms.getCluster([5.8,2.7,3.2,0.9])

1